# Chapter 3 - Exercises

> Author : Badr TAJINI - Large Language model (LLMs) - ESIEE 2024-2025

---

# Exercise 3.1

Observe that the `nn.Linear` layer in `SelfAttention_v2` employs a distinct weight initialization strategy compared to the `nn.Parameter(torch.rand(d_in, d_out))` method utilized in `SelfAttention_v1`, resulting in divergent computational outputs. To validate the fundamental structural similarities between the two implementations, we propose a weight transfer methodology that will demonstrate the potential for convergence between `SelfAttention_v1` and `SelfAttention_v2`.

**Key Exercise Question: Can you transfer the weights from `SelfAttention_v2` to `SelfAttention_v1` such that both implementations produce identical output tensors?**

*Specific Challenges:*
- Recognize that `nn.Linear` stores its weight matrix in a transposed configuration
- Carefully map and transfer weights between the two self-attention implementations
- Verify that the transferred weights result in mathematically equivalent computational results

The primary objective is to systematically transfer weight matrices from an instantiated `SelfAttention_v2` object to a `SelfAttention_v1` instance, requiring a nuanced understanding of the underlying weight matrix representation.

Subsequent research focuses on advancing the self-attention mechanism through two critical architectural enhancements:

1. **Causal Masking**: This modification introduces a constraint preventing the attention mechanism from accessing future sequence elements. Such a constraint is particularly pivotal in generative language modeling contexts, where each token's prediction must be conditioned exclusively on preceding contextual information.

2. **Multi-Head Attention**: This approach involves partitioning the attention mechanism into parallel computational "heads." Each head operates as a distinct learnable feature extractor, capable of capturing diverse representational characteristics across different subspaces and positional contexts. By enabling simultaneous multi-perspective representation learning, this technique substantially augments the model's capacity to process complex, high-dimensional representations.

These architectural refinements collectively contribute to more sophisticated and contextually aware neural network architectures, particularly in sequence modeling domains.

### Résultats - Exercise 3.1

In [ ]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

d_in, d_out = 3, 2

In [42]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)

In [43]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=False)
        self.W_key   = nn.Linear(d_in, d_out, bias=False)
        self.W_value = nn.Linear(d_in, d_out, bias=False)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v2 = SelfAttention_v2(d_in, d_out)

In [44]:
sa_v1.W_query = torch.nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_key = torch.nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_value = torch.nn.Parameter(sa_v2.W_value.weight.T)

In [45]:
sa_v1(inputs)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

In [46]:
sa_v2(inputs)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

**Interprétation**

Ici, on copie les poids de v2 vers v1 (avec une transposition) pour s’assurer que les deux modèles utilisent exactement les mêmes matrices. En résultat on a donc les sorties qui deviennent identiques.  

# Exercise 3.2

**Key Exercise Question: How can you modify the input arguments to the `MultiHeadAttentionWrapper(num_heads=2)` to transform the output context vectors from four-dimensional to two-dimensional while maintaining the `num_heads=2` configuration?**

*Specific Challenges:*
- Identify the input parameter that controls the dimensionality of output context vectors
- Understand the relationship between input arguments and tensor shape
- Achieve dimensionality reduction without modifying the core `MultiHeadAttentionWrapper` class implementation

*Architectural Context:*
Up to this point, we have developed a `MultiHeadAttentionWrapper` that integrates multiple single-head attention modules through sequential processing, implemented via the comprehension `[head(x) for head in self.heads]` in the forward method. This current implementation represents a foundational approach to multi-head attention mechanisms.

*Potential Optimization Strategies:*
1. **Sequential Processing Limitation**: The current implementation processes attention heads sequentially, which may introduce computational inefficiencies.

2. **Parallel Processing Approach**: An advanced optimization involves simultaneous computation of attention head outputs through efficient matrix multiplication techniques. This parallel processing strategy can potentially enhance computational performance and reduce computational overhead.

*Theoretical Implications:*
The ability to dynamically adjust output dimensionality while maintaining the multi-head attention structure highlights the flexibility of modern neural network architectural designs. Such manipulations are crucial in adapting attention mechanisms to diverse computational requirements across different machine learning domains.

*Practical Recommendation:*
Carefully examine the input arguments of the `MultiHeadAttentionWrapper` and consider how specific parameters might influence the output tensor's dimensionality. The solution likely involves a subtle adjustment that does not require restructuring the core implementation.

### Résultats - Exercise 3.2

In [47]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [48]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

In [49]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 1
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.5740,  0.2216],
         [-0.7320,  0.0155],
         [-0.7774, -0.0546],
         [-0.6979, -0.0817],
         [-0.6538, -0.0957],
         [-0.6424, -0.1065]],

        [[-0.5740,  0.2216],
         [-0.7320,  0.0155],
         [-0.7774, -0.0546],
         [-0.6979, -0.0817],
         [-0.6538, -0.0957],
         [-0.6424, -0.1065]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


**Interprétation**

Le paramètre qui contrôle la dimension de sortie est d_out (la taille des vecteurs produits par chaque tête).

Comme MultiHeadAttentionWrapper concatène les sorties des têtes, la dimension finale vaut :
dim_finale = num_heads × d_out

Donc :
- si num_heads = 2
- et qu’on veut une sortie en 2 dimensions

il suffit de choisir d_out = 1.

# Exercise 3.3

**Key Exercise Question: Can you configure a `MultiHeadAttention` module that precisely replicates the architectural specifications of the smallest GPT-2 model?**

*Specific Model Specifications:*
- Number of Attention Heads: 12
- Input/Output Embedding Dimensions: 768
- Context Length: 1,024 tokens

*Architectural Parameters:*
- `num_heads`: 12
- `d_model`: 768
- `context_length`: 1,024

*Theoretical Considerations:*
The proposed configuration mirrors the smallest variant of the GPT-2 model, which represents a fundamental architecture in transformer-based language models. By precisely replicating these specifications, we can explore the intricate design choices that contribute to the model's effectiveness in natural language processing tasks.

*Key Implementation Details:*
- Ensuring 12 parallel attention heads allows for multi-perspective feature representation
- The 768-dimensional embedding space provides a rich, high-dimensional representation of linguistic context
- The 1,024 token context length enables comprehensive sequence processing

*Practical Recommendation:*
Carefully construct the `MultiHeadAttention` initialization to match these exact specifications, paying close attention to the dimensionality and number of heads to accurately reproduce the smallest GPT-2 model's architectural characteristics.

### Résultats - Exercise 3.3

In [50]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [51]:
context_length = 1024
d_in, d_out = 768, 768
num_heads = 12

mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads)

In [52]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(mha)

2360064

**Interprétation**

Ici on configure une couche MultiHeadAttention pour correspondre aux paramètres du GPT-2 Small (124M) :

- context_length = 1024
- d_in = 768
- d_out = 768
- num_heads = 12

La fonction count_parameters permet ensuite de vérifier le nombre de paramètres entraînables dans cette couche, ce qui est utile pour comparer avec la complexité attendue du modèle qui est de 177 millions en tout.